# Remembered Object Position Task - Analysis - Part 2 - LMMs

## Data Import

In [ ]:
# Load libraries
library('tidyverse')

library('lme4')
library('pwr')
library('emmeans')
library('parameters')
library('sjstats')
library('sjPlot')
library('lmerTest')
library('ggeffects')
library('buildmer')


# Folders
dir.data <- '../results'
dir.out <- dir.data


In [ ]:
install.packages('afex')

In [ ]:
# Import datasets
trials <- read_tsv(paste(dir.data, '/lmm_analysis_trials.csv', sep='')) # Trial responses
trials_var <- read_tsv(paste(dir.data, '/lmm_analysis_stddev.csv', sep='')) # Aggregated SD values
tlx <- read_tsv(paste(dir.data, '/lmm_analysis_tlx.csv', sep='')) # NASA TLX scores

# Set factors
trials$response_mode <- factor(trials$response_mode, levels=c('object', 'cube', 'fixed', 'laser'))
trials$session_num <- factor(trials$session_num, levels=c(1, 2, 3, 4), order=T)
trials_var$response_mode <- factor(trials_var$response_mode, levels=c('object', 'cube', 'fixed', 'laser'))
trials_var$session_num <- factor(trials_var$session_num, levels=c(1, 2, 3, 4), order=T)
tlx$response_mode <- factor(tlx$response_mode, levels=c('object', 'cube', 'fixed', 'laser'))
tlx$session_num <- factor(tlx$session_num, levels=c(1, 2, 3, 4), order=T)


## Model Selection


In [ ]:
# Terms to include in all models evaluated by buildmer
force.include <- 'response_mode + session_num + response_mode:session_num + (1|ppid)'

### Accuracy

In [ ]:
# Accuracy
max.acc <- tar_error_pos ~ response_mode + session_num + response_mode:session_num + (1 + response_mode + session_num|ppid) + (1+ response_mode + session_num|target)
#m.acc <- buildmer(max.acc, trials, buildmerControl=buildmerControl(include=force.include))

#print(formula(m.acc@model))
#print(m.acc)

# 0. buildmer ends on:
# tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid) + (1 + response_mode | target)
# REML criterion at convergence: -12858.88

# 1. Check if response_mode | ppid improves fit
#m0 <- lmer(data=trials, tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid) + (1 + response_mode | target), REML=F)
#m1 <- lmer(data=trials, tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 + response_mode | ppid) + (1 + response_mode | target), REML=F)
#print(anova(m0, m1))

# Result: Model with response_mode | ppid fits better
#    npar    AIC    BIC logLik deviance  Chisq Df Pr(>Chisq)    
# m0   28 -12968 -12800 6511.8   -13024                         
# m1   37 -13120 -12899 6597.0   -13194 170.49  9  < 2.2e-16 ***
# Note on negative logLik: https://stat.ethz.ch/pipermail/r-help/2005-April/069550.html


# 2. Check if including session_num | ppid improves fit
#m2 <- lmer(data=trials, tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 + response_mode + session_num | ppid) + (1 + response_mode | target), REML=F)

# Result: m2 does not converge


# 3. Check if session_num | target improves fit
#m3 <- lmer(data=trials, tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 + response_mode | ppid) + (1 + response_mode + session_num | target), REML=F)

# Result: m3 does not converge


# FINAL model for Accuracy: 
f.acc <- tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 + response_mode | ppid) + (1 + response_mode | target)


### Precision

In [ ]:
# Precison (pre-aggregated by session)
max.prec <- tar_error_pos ~ response_mode + session_num + response_mode:session_num + (1 + response_mode + session_num | ppid)
#m.prec <- buildmer(max.prec, trials_var, buildmerControl=buildmerControl(include=force.include))

#print(formula(m.prec@model))

# 0. buildmer ends on:
# tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid)

# For all additional RE terms, observations <= random effects, no convergence

# FINAL model for Precision
f.prec <- tar_error_pos ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid)


### Response Duration

In [ ]:
# Response Duration
max.rt <- response_duration ~ response_mode + session_num + response_mode:session_num + (1 +  response_mode + session_num|ppid) + (1+ response_mode + session_num|target)
#m.rt <- buildmer(max.rt, trials, buildmerControl=buildmerControl(include=force.include))

#print(formula(m.rt@model))

# 0. buildmer ends on:
# response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid) + (1 | target)


# 1. Check response_mode | ppid
m0 <- lmer(data=trials, response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid) + (1 | target), REML=F)
#m1 <- lmer(data=trials, response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 + response_mode | ppid) + (1 | target), REML=F)

# Result: m1 does not converge


# 2. Check session_num | ppid
m2 <- lmer(data=trials, response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 + session_num | ppid) + (1 | target), REML=F)
#print(anova(m0, m2))

# Result: Model with session_num | ppid fits better
# m0   19 11146 11260 -5553.9    11108                         
# m2   28 10529 10696 -5236.5    10473 634.78  9  < 2.2e-16 ***

# 3. Check if session_num | target further improves fit
#m3 <- lmer(data=trials, response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 + session_num | ppid) + (1 + session_num | target), REML=F)

# Result: m3 does not converge.

# 4. Check if response_mode | target further improves fit
#m4 <- lmer(data=trials, response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 + session_num| ppid) + (1 + response_mode | target), REML=F)
#print(anova(m4, m2))

# Result: It does. 
# m2   28 10529 10696 -5236.5    10473                         
# m4   37 10439 10660 -5182.5    10365 107.94  9  < 2.2e-16 ***

# 5. Either adding "+ response_mode | ppid" or "+ session_num | target" fail to converge

# FINAL model for response duration: 
f.rt <- response_duration ~ 1 + response_mode + session_num + response_mode:session_num + (1 + session_num| ppid) + (1 + response_mode | target)


### TLX Score

In [ ]:
# Subjective Workload
max.tlx <- Global ~ response_mode + session_num + response_mode:session_num + (1 + response_mode + session_num|ppid)
#m.tlx <- buildmer(max.tlx, tlx, buildmerControl=buildmerControl(include=force.include))
#print(formula(m.tlx@model))

# 0. buildmer ends on:
# Global ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid)

# For all additional RE terms, observations <= random effects, no convergence

# FINAL model for TLX score
f.tlx <- Global ~ 1 + response_mode + session_num + response_mode:session_num + (1 | ppid)

## Final Models

### Accuracy

In [ ]:
# Fit final model
model.acc <- lmer(data=trials, f.acc)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics:\n')
print(summary(model.acc))
print(anova(model.acc))


In [ ]:
print(eta_squared(anova(model.acc)))

In [ ]:
# Post-hoc contrasts
cat('\n\n* Post-Hoc Comparisons for Response Mode:\n')
emm_response <- emmeans(model.acc, pairwise ~ response_mode, adjust="holm")
print(emm_response$contrasts)
print(eff_size(emm_response, sigma = sigma(model.acc), edf = df.residual(model.acc)))

cat('\n\n* Post-Hoc Comparisons for Session:\n')
print(emmeans(model.acc, pairwise ~ session_num, adjust="holm")$contrasts)


In [ ]:
# emmeans for plot
cat('\n\n* Estimated Marginal Means for Response:\n')
gemm.response <- ggemmeans(model.acc, terms=c('response_mode'))
write.csv(gemm.response, paste(dir.out, 'emmeans_acc_response.csv', sep='/'), row.names = FALSE)
gemm.response

cat('\n\n* Estimated Marginal Means for Session:\n')
gemm.session <- ggemmeans(model.acc, terms=c('session_num'))
write.csv(gemm.session, paste(dir.out, 'emmeans_acc_session.csv', sep='/'), row.names = FALSE)
gemm.session


In [ ]:
# Effects plots
plot_model(model.acc, type='pred')

### Precision

In [ ]:
# Fit final model
model.prec <- lmer(data=trials_var, f.prec)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics:\n')
print(summary(model.prec))
print(anova_stats(model.prec))


In [ ]:
# Post-hoc contrasts
cat('\n\n* Post-Hoc Comparisons for Response Mode:\n')
emm_response <- emmeans(model.prec, pairwise ~ response_mode, adjust="holm")
print(emm_response$contrasts)
print(eff_size(emm_response, sigma = sigma(model.prec), edf = df.residual(model.prec)))

cat('\n\n* Post-Hoc Comparisons for Session:\n')
print(emmeans(model.prec, pairwise ~ session_num, adjust="holm")$contrasts)


In [ ]:
# emmeans for plot
cat('\n\n* Estimated Marginal Means for Response:\n')
gemm.response <- ggemmeans(model.prec, terms=c('response_mode'))
write.csv(gemm.response, paste(dir.out, 'emmeans_prec_response.csv', sep='/'), row.names = FALSE)
gemm.response

cat('\n\n* Estimated Marginal Means for Session:\n')
gemm.session <- ggemmeans(model.prec, terms=c('session_num'))
write.csv(gemm.session, paste(dir.out, 'emmeans_prec_session.csv', sep='/'), row.names = FALSE)
gemm.session


In [ ]:
# Effects plots
plot_model(model.prec, type='pred')

### Response Duration

In [ ]:
# Fit final model
model.rt <- lmer(data=trials, f.rt)

# Model + Anova results
cat('* ANOVA Omntrials_varsults + Statistics:\n')
print(summary(model.rt))
print(anova_stats(model.rt))


In [ ]:
# Post-hoc contrasts
cat('\n\n* Post-Hoc Comparisons for Response Mode:\n')
emm_response <- emmeans(model.rt, pairwise ~ response_mode, adjust="holm")
print(emm_response$contrasts)
print(eff_size(emm_response, sigma = sigma(model.rt), edf = df.residual(model.rt)))

cat('\n\n* Post-Hoc Comparisons for Session:\n')
print(emmeans(model.rt, pairwise ~ session_num, adjust="holm")$contrasts)


In [ ]:
# emmeans for plot
cat('\n\n* Estimated Marginal Means for Response:\n')
gemm.response <- ggemmeans(model.rt, terms=c('response_mode'))
write.csv(gemm.response, paste(dir.out, 'emmeans_rt_response.csv', sep='/'), row.names = FALSE)
gemm.response

cat('\n\n* Estimated Marginal Means for Session:\n')
gemm.session <- ggemmeans(model.rt, terms=c('session_num'))
write.csv(gemm.session, paste(dir.out, 'emmeans_rt_session.csv', sep='/'), row.names = FALSE)
gemm.session


In [ ]:
# Effects plots
plot_model(model.rt, type='pred')

### TLX Score

In [ ]:
# Fit final model
model.tlx <- lmer(data=tlx, f.tlx)

# Model + Anova results
cat('* ANOVA Omntrials_varsults + Statistics:\n')
print(summary(model.tlx))
print(anova_stats(model.tlx))


In [ ]:
# Post-hoc contrasts
cat('\n\n* Post-Hoc Comparisons for Response Mode:\n')
emm_response <- emmeans(model.tlx, pairwise ~ response_mode, adjust="holm")
print(emm_response$contrasts)
print(eff_size(emm_response, sigma = sigma(model.tlx), edf = df.residual(model.tlx)))

cat('\n\n* Post-Hoc Comparisons for Session:\n')
print(emmeans(model.tlx, pairwise ~ session_num, adjust="holm")$contrasts)
print(eff_size(emmeans(model.tlx, pairwise ~ session_num, adjust="holm"), sigma = sigma(model.tlx), edf = df.residual(model.tlx)))

In [ ]:
# emmeans for plot
cat('\n\n* Estimated Marginal Means for Response:\n')
gemm.response <- ggemmeans(model.tlx, terms=c('response_mode'))
write.csv(gemm.response, paste(dir.out, 'emmeans_tlx_response.csv', sep='/'), row.names = FALSE)
gemm.response

cat('\n\n* Estimated Marginal Means for Session:\n')
gemm.session <- ggemmeans(model.tlx, terms=c('session_num'))
write.csv(gemm.session, paste(dir.out, 'emmeans_tlx_session.csv', sep='/'), row.names = FALSE)
gemm.session


In [ ]:
# Plot effects
plot_model(model.tlx, type='pred')